<a href="https://colab.research.google.com/github/bill7845/brunch_networking_aws/blob/main/Notebook/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Colab에서 Konlpy 사용
"""
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy

!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab/

In [ ]:
! bash install_mecab-ko_on_colab190912.sh

In [ ]:
"""
AWS Setting
"""
!pip install boto3
!pip install awscli

In [ ]:
## library import
import pandas as pd
import numpy as np
import json
import os
import pickle
import urllib.request
import re
from ast import literal_eval
from tqdm.notebook import tqdm
from google.oauth2 import service_account
from google.cloud import storage

import boto3
import os
import io

from konlpy.tag import Okt
from konlpy.tag import Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
"""
aws auth setting
"""
!aws configure

In [ ]:
"""
bigquery Authentication setting.
"""

bq_credential_path = '/content/drive/MyDrive/Colab code/brunch_project/code/gcp_access/brunch-networking-07958d4e3d41.json'
credentials = service_account.Credentials.from_service_account_file(bq_credential_path)
project_id = 'brunch-networking-303012'

In [ ]:
def get_data_from_bq(credentials=credentials, project_id=project_id, class_num=int):
  
  if class_num == None:
    
    query = "SELECT * FROM `brunch-networking-303012.brunch_networking.ori_brunch_data`"
    df = pd.read_gbq(query=query, project_id=project_id, credentials=credentials, dialect='standard')
    
  elif class_num != None:
    query = "SELECT * FROM `brunch-networking-303012.brunch_networking.ori_brunch_data` where class_num = {class_num}".format(class_num=class_num)
    df = pd.read_gbq(query=query, project_id=project_id, credentials=credentials, dialect='standard')

  df = df[~df['text'].isnull()] # delete empty text row

  return df

In [ ]:
def s3_upload(bucket_name , bucket_key, target_file_path):
  
  s3 = boto3.client('s3')
  
  s3.upload_file(
      Filename = target_file_path,
      Bucket = bucket_name,
      Key = bucket_key
  )

In [ ]:
def to_matrix_eachCategory(class_num, df):

  mecab = Mecab()
  
  text = df['text']
  text_plat = text.values.tolist()
  
  label = str(class_num)

  prepared_data = []
  tmp = None
  for idx in range(len(text_plat)):
      tmp = [i[0] for i in mecab.pos(text_plat[idx]) if ( ((i[1]=="NNG") or (i[1]=="NNP")) and(len(i[0])>1))] # 품사가 명사이면서, 길이가 2이상
      prepared_data.append(" ".join(tmp))

  tfidf_vect = TfidfVectorizer(max_df=0.9)
  tfidf_vect.fit(prepared_data)
  tfidf_matrix = tfidf_vect.transform(prepared_data)

  path_vect = '/content/drive/My Drive/Colab code/brunch_project/data/vect/each_vect/'
  pickle.dump(tfidf_vect, open(os.path.join(path_vect,label+'tfidf_vect.pkl'),'wb'),protocol=4)

  s3_upload(bucket_name='util-brunch-networking' , bucket_key='each_vect/' + label + "tfidf_vect.pkl", target_file_path=path_vect + label + 'tfidf_vect.pkl')

  path_matrix = '/content/drive/My Drive/Colab code/brunch_project/data/vect/each_matrix/'
  pickle.dump(tfidf_matrix, open(os.path.join(path_matrix,label+'tfidf_matrix.pkl'),'wb'),protocol=4)

  s3_upload(bucket_name='util-brunch-networking' , bucket_key='each_matrix/' + label + "tfidf_matrix.pkl", target_file_path=path_matrix + label + 'tfidf_matrix.pkl')

  return tfidf_matrix

In [ ]:
def keyword_vectorize(df):
  
  df = df[~pd.isnull(df['keyword'])]
  df.loc[:,"keyword"] = df["keyword"].apply(lambda x : literal_eval(str(x)))

  df['keyword_literal'] = df['keyword'].apply(lambda x : (' ').join(x)) # keyword vector => ["keyword1, keyword2, keyword3.."]

  keyword_count_vect = CountVectorizer() 
  keyword_mat = keyword_count_vect.fit_transform(df['keyword_literal']) # CountVectorizer로 벡터화


  path_keyword_vect = '/content/drive/My Drive/Colab code/brunch_project/data/vect/keyword/'

  pickle.dump(keyword_count_vect, open(os.path.join(path_keyword_vect,'keyword_count_vect.pkl'),'wb'),
              protocol=4)
  s3_upload(bucket_name='util-brunch-networking',
            bucket_key='keyword_vect/' + "keyword_count_vect.pkl",
            target_file_path=path_keyword_vect + 'keyword_count_vect.pkl')


  pickle.dump(keyword_mat, open(os.path.join(path_keyword_vect,'keyword_mat.pkl'),'wb'),
              protocol=4)
  s3_upload(bucket_name='util-brunch-networking',
            bucket_key='keyword_matrix/' + "keyword_mat.pkl",
            target_file_path=path_keyword_vect + 'keyword_mat.pkl')

In [ ]:
## fixed range 0~17
for i in range(0,18):
  print("class ", i)
  df = get_data_from_bq(class_num=i)
  to_matrix_eachCategory(i,df)

all_df = pd.read_csv("/content/drive/MyDrive/Colab code/brunch_project/data/basement/all_df.csv") # load all_df.csv from google drive
keyword_vectorize(df=all_df)